<a href="https://colab.research.google.com/github/Jessica1842553/MCDdm2026/blob/main/Tarea3_DM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tarea 3 - **PySpark ML**
### Jessica Lizeth Hernández Bracho - 1842553

El dataset MovieLens 10M contiene aproximadamente 10 millones de calificaciones realizadas por usuarios a diferentes películas.

**El objetivo es construir un sistema de recomendación que prediga el rating que un usuario daría a una película.**

### Instalacion de paquetes

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,901 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,742 kB]
Get:14 http://

### Creación de la sesión

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
               .appName('ml-t3') \
               .getOrCreate()

### Carga de datos

In [3]:
# Descarga de MovieLens 10M dataset
!wget -q --show-progress http://files.grouplens.org/datasets/movielens/ml-10m.zip -P /content/
!unzip -q /content/ml-10m.zip -d /content/

ratings = spark.read \
    .option("delimiter", "::") \
    .option("header", "false") \
    .csv(
        "/content/ml-10M100K/ratings.dat",
        schema="userId INT, movieId INT, rating DOUBLE, timestamp LONG"
    )

movies = spark.read \
    .option("delimiter", "::") \
    .option("header", "false") \
    .csv(
        "/content/ml-10M100K/movies.dat",
        schema="movieId INT, title STRING, genres STRING"
    )

tags = spark.read \
    .option("delimiter", "::") \
    .option("header", "false") \
    .csv(
        "/content/ml-10M100K/tags.dat",
        schema="userId INT, movieId INT, tag STRING, timestamp LONG"
    )

ml-10m.zip          100%[===================>]  62.53M  52.3MB/s    in 1.2s    


Exploración de los datos - Se organiza en tres archivos principales:

***ratings.dat:*** Contiene las calificaciones que los usuarios dieron a las películas.
Incluye el identificador único del usuario (userId), el identificador de la película (movieId), la calificación que el usuario asignó (rating) y una marca de tiempo (timestamp) indicando cuándo se realizó la calificación.

 - *Las calificaciones van de 0.5 a 5.0 estrellas.*

***movies.dat:*** Contiene información de las películas. Incluye el identificador de la película (movieId), el título de la película (title) y sus géneros (genres).

***tags.dat:*** Contiene etiquetas opcionales que los usuarios agregaron a las películas. Incluye el identificador del usuario (userId), el identificador de la película (movieId), el texto de la etiqueta (tag) y la marca de tiempo (timestamp).

In [4]:
ratings = ratings.toDF("userId", "movieId", "rating", "timestamp")
ratings.printSchema()
ratings.show(5)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|    122|   5.0|838985046|
|     1|    185|   5.0|838983525|
|     1|    231|   5.0|838983392|
|     1|    292|   5.0|838983421|
|     1|    316|   5.0|838983392|
+------+-------+------+---------+
only showing top 5 rows


In [8]:
ratings.count() # total de registros
ratings.select("userId").distinct().count() # conteo de usuarios
ratings.select("movieId").distinct().count() #conteo de peliculas

ratings.groupBy("rating").count().orderBy("rating").show() # distribución del rating
ratings.dropna().count()

+------+-------+
|rating|  count|
+------+-------+
|   0.5|  94988|
|   1.0| 384180|
|   1.5| 118278|
|   2.0| 790306|
|   2.5| 370178|
|   3.0|2356676|
|   3.5| 879764|
|   4.0|2875850|
|   4.5| 585022|
|   5.0|1544812|
+------+-------+



10000054

### Preprocesamiento

In [9]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)



In [11]:
movies.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [10]:
tags.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: long (nullable = true)



### Modelo ALS - Alternating Least Squares

ALS hace factorización de matrices, básicamente calcula un producto punto entre vectores latentes de usuario y película: $r^ui​=Uu​⋅Vi​$

Explicación:
- $r^ui$ → rating predicho para el usuario 𝑢 sobre la película 𝑖
- $U𝑢$ → vector latente del usuario 𝑢
- $Vi​$ → vector latente de la película 𝑖
- El producto punto $Uu​⋅Vi$ genera el rating estimado

Es un método de factorización de matrices que:

- Aprende vectores latentes para usuarios

- Aprende vectores latentes para películas

- Predice rating como producto punto de esos vectores

El modelo aprende “gustos ocultos” de los usuarios y “características ocultas” de las películas.

In [12]:
# Dividir datos en entrenamiento y prueba
train, test = ratings.randomSplit([0.7, 0.3], seed=42)

In [13]:
from pyspark.ml.recommendation import ALS

In [14]:
als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    rank=10, # número de factores latentes
    maxIter=10, # iteraciones
    regParam=0.1, #regularización sobreajuste
    coldStartStrategy="drop"
)

model = als.fit(train)

### Evaluación

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator

In [16]:
predictions = model.transform(test)

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print("RMSE:", rmse)

RMSE: 0.8149616906947948


RMSE mide el error promedio entre el rating real y el predicho.

**El modelo obtuvo un RMSE de 0.8149**, lo cual indica que en promedio el error de predicción es menor a una estrella.

### Comparación - Promedio global del rating

In [24]:
from pyspark.sql.functions import avg

promg_mean = train.select(avg("rating")).first()[0]
print("Promedio global:", promg_mean)

Promedio global: 3.5129322583409506


In [26]:
from pyspark.sql.functions import lit

pg_predic = test.withColumn("predic", lit(promg_mean))
pg_predic.show(5)

+------+-------+------+---------+------------------+
|userId|movieId|rating|timestamp|            predic|
+------+-------+------+---------+------------------+
|     1|    231|   5.0|838983392|3.5129322583409506|
|     1|    355|   5.0|838984474|3.5129322583409506|
|     1|    362|   5.0|838984885|3.5129322583409506|
|     1|    364|   5.0|838983707|3.5129322583409506|
|     1|    466|   5.0|838984679|3.5129322583409506|
+------+-------+------+---------+------------------+
only showing top 5 rows


In [28]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="predic"
)

pg_rmse = evaluator.evaluate(pg_predic)
print("Prom Global RMSE:", pg_rmse)

Prom Global RMSE: 1.0611575682868193


Para evaluar la calidad del modelo, se comparó contra el promedio global del rating.

- El promedio global obtuvo un RMSE de 1.0611.
- El modelo ALS obtuvo un RMSE de 0.8150, reduciendo el error aproximadamente en un 23%.

Esto demuestra que el modelo logra capturar patrones reales de preferencia entre usuarios y películas.

In [22]:
# El modelo puede generar las 5 películas que más probablemente le gusten a cada usuario.
user_recs = model.recommendForAllUsers(5)
user_recs.show(5, False)

+------+-------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                  |
+------+-------------------------------------------------------------------------------------------------+
|1     |[{65133, 6.838792}, {6511, 6.058783}, {26842, 6.048556}, {60046, 6.0179133}, {32090, 6.0121593}] |
|3     |[{65133, 5.196769}, {61475, 4.998976}, {34299, 4.9514675}, {32444, 4.889678}, {32460, 4.8659782}]|
|5     |[{61742, 4.999383}, {64275, 4.859806}, {41627, 4.7822237}, {64408, 4.7607985}, {53883, 4.739488}]|
|6     |[{32657, 5.3015876}, {4454, 5.184665}, {53883, 5.137222}, {527, 5.003424}, {318, 5.0013924}]     |
|9     |[{53883, 5.3647566}, {6439, 5.0924664}, {58898, 5.039753}, {4454, 5.002769}, {64275, 4.925533}]  |
+------+-------------------------------------------------------------------------------------------------+
only showing top 5 rows


In [21]:
from pyspark.sql.functions import explode

model.recommendForAllUsers(5) \
    .withColumn("rec", explode("recommendations")) \
    .select("userId", "rec.movieId", "rec.rating") \
    .join(movies, on="movieId") \
    .select("userId", "title", "rating") \
    .show(10, False)

+------+----------------------------------------------+---------+
|userId|title                                         |rating   |
+------+----------------------------------------------+---------+
|1     |Blackadder Back & Forth (1999)                |6.838792 |
|1     |Jesus (1979)                                  |6.058783 |
|1     |Tai Chi Master (Tai ji: Zhang San Feng) (1993)|6.048556 |
|1     |Children of Huang Shi, The (2008)             |6.0179133|
|1     |Low Life, The (1995)                          |6.0121593|
|3     |Blackadder Back & Forth (1999)                |5.196769 |
|3     |August Evening (2007)                         |4.998976 |
|3     |Appaloosa, The (1966)                         |4.9514675|
|3     |Carmen (1983)                                 |4.889678 |
|3     |Knockin' On Heaven's Door (1997)              |4.8659782|
+------+----------------------------------------------+---------+
only showing top 10 rows
